# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features['B']
logdis = np.log(boston_features['DIS'])
loglstat = np.log(boston_features['LSTAT'])

# Min-Max scaling
boston_features['B'] = (b-min(b))/(max(b)-min(b))
boston_features['DIS'] = (logdis-min(logdis))/(max(logdis)-min(logdis))

# Standardization
boston_features['LSTAT'] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target, columns = ['target'])
type(X)

pandas.core.frame.DataFrame

### Train-test split

Perform a train-test split with a test set of 20%.

In [3]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

In [4]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=51)

In [5]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [6]:
linreg.fit(X, y)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [7]:
test_residuals = y_hat_test - y_test
print(test_residuals)

        target
389   3.063393
428   2.157391
162   0.082762
132   0.529613
270  -2.720772
323   0.284355
203 -10.826401
23   -0.406198
213  -4.250459
316  -2.492471
200  -0.526593
186 -12.718342
29    1.064635
382   2.722562
477  -0.682961
258  -4.919036
358   2.269211
397   7.877368
500   2.836783
106  -2.945448
495  -7.188221
349   1.860400
256  -6.250975
480  -0.494833
425   1.525227
169   0.571645
447   6.632665
214 -15.280997
384  -2.552078
342   7.526338
..         ...
245  -6.253566
470  -1.655350
365  -8.337166
445   0.440683
235  -2.580854
442   0.967283
68   -0.959081
204  -8.815878
264  -6.259481
287   1.338599
345   1.937734
280  -7.527089
126  -3.649683
369  -9.715734
427   5.578138
101   1.851608
339   2.696313
77    1.211062
39    1.125380
153  -2.254897
364  20.768476
435   0.337257
493  -1.291244
207  -7.739734
279  -2.855478
58    1.179892
409  -9.458446
206  -2.517195
353  -0.515999
160   6.542730

[102 rows x 1 columns]


In [8]:
from sklearn.metrics import mean_squared_error

test_mse = mean_squared_error(y_test, y_hat_test)
print(test_mse)

25.717986859416037


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [21]:
def kfolds(data, k):
    df = pd.DataFrame(data=data)
    df_split = np.array_split(df, k)
    return df_split

### Apply it to the Boston Housing data

In [22]:
# Make sure to concatenate the data again
bos_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [23]:
# Apply kfolds() to bos_data with 5 folds
bos_data_folded = kfolds(bos_data, 5)

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [24]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
from sklearn.metrics import mean_squared_error

In [27]:
test_errs = []
train_errs = []
k = 5

for n in range(k):

    # Split in train and test for the fold
    test = bos_data_folded[n]
    train = pd.concat([fold for i, fold in enumerate(bos_data_folded) if i!=n])

    # Fit a linear regression model
    linreg.fit(train[X.columns], train.target)
    y_hat_train = linreg.predict(train[X.columns])
    y_hat_test = linreg.predict(test[X.columns])

    # Evaluate Train and Test errors
    test_errs.append(mean_squared_error(test.target, y_hat_test))
    train_errs.append(mean_squared_error(train.target, y_hat_train))

print(train_errs)
print(test_errs)

[24.195577370388616, 23.032087348477972, 19.74507285797898, 15.31710138425119, 22.32997280754659]
[13.4051449220082, 17.44401680068204, 37.03271139002073, 58.2795438478423, 26.09798875714838]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [29]:
from sklearn.model_selection import cross_val_score

cv_5_result = cross_val_score(linreg, X, y, cv=5, scoring='neg_mean_squared_error')
print(cv_5_result)

[-13.40514492 -17.4440168  -37.03271139 -58.27954385 -26.09798876]


Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [30]:
print(cv_5_result)

[-13.40514492 -17.4440168  -37.03271139 -58.27954385 -26.09798876]


##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 